In [1]:
# Importações
import nltk
from nltk.corpus import stopwords
import pandas as pd
import re
from sklearn.utils import shuffle
import csv

# Caso o NLTK não esteja configurado:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/alice.ferreira/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/alice.ferreira/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alice.ferreira/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Importa os dados extraídos
df_true = pd.read_csv('dados/df_combinedtrue.csv')
df_false = pd.read_csv('dados/df_combinedfake.csv')

In [3]:
# Carrega o FakeRecogna
df_fakerecogna = pd.read_excel('dados/FakeRecogna.xlsx')

# Filtra apenas as notícias de saúde
df_fakerecogna = df_fakerecogna.loc[df_fakerecogna["Categoria"] == 'saúde']

df_fakerecogna = df_fakerecogna.drop(['Categoria', 'Subtitulo', 'Autor'], axis=1)
df_fakerecogna = df_fakerecogna[['Titulo', 'URL', 'Data', 'Classe', 'Noticia']]
df_fakerecogna = df_fakerecogna.rename(columns = {'Titulo':'title', 'URL': 'link', 'Data': 'date', 'Classe':'classe','Noticia':'corpo_texto'})
df_fakerecogna['classe'] = df_fakerecogna['classe'].astype('int64')

display(df_fakerecogna)

,title,link,date,classe,corpo_texto
1,Equador prepara cova coletiva para mortos por ...,https://noticias.uol.com.br/internacional/ulti...,27/03/2020 18h25,1,o governar equador anunciar preparar cova cole...
2,Air France voltará a operar voo direto Pequim-...,https://www.uol.com.br/nossa/noticias/afp/2020...,07/08/2020 13h42,1,o companhia air france operar voar direto pequ...
3,Marfrig intensifica venda de carne do Brasil a...,https://economia.uol.com.br/noticias/reuters/2...,27/04/2020 14h53,1,o marfrig global foods retomar vender carnar b...
6,Coronavírus: sindicatos dos inquilinos convoca...,https://noticias.uol.com.br/internacional/ulti...,30/03/2020 10h38,1,greve pagamento aluguel organizar sindicato in...
9,\n\nVacina contra Covid-19 faz imunidade do co...,https://www.boatos.org/saude/vacina-contra-cov...,27/04/2021,0,“ reação vacinar entender facilidade pessoa in...
...,...,...,...,...,...
11891,\n\nVacinas estão sendo aplicadas com seringas...,https://www.boatos.org/saude/vacinas-aplicadas...,02/01/2021,0,atenção brasil simulação agulhar líquido conse...
11892,Papa vai retomar as audiências públicas das qu...,https://g1.globo.com/mundo/noticia/2020/08/26/...,26/08/2020 08h41,1,o papar francisco retomar audiência público qu...
11899,Ministro se encontra com representantes da ONU...,https://www.gov.br/saude/pt-br/assuntos/notici...,19/04/2021 19h13,1,o ministrar saudar marcelo queiroga e represen...
11900,vídeo mostrar garoto afogar ninguém nado,https://www.e-farsas.com/um-video-mostra-um-ga...,7/05/20197,0,o vídeo recentemente publicar página “ 18:18 ”...


In [4]:
# Palavras a serem buscadas
keywords = ["vacina", "vacinação", "vacinacao", "covid", "pandemia", "corona", "coronavírus", "coronavirus", "quarentena", "sars-cov-2"]

In [5]:

# filtra o dataset com o mesmo critério que o de extração
df_fakerecogna = df_fakerecogna[df_fakerecogna['title'].str.contains('|'.join(keywords), case=False, na=False)]

display(df_fakerecogna)

,title,link,date,classe,corpo_texto
1,Equador prepara cova coletiva para mortos por ...,https://noticias.uol.com.br/internacional/ulti...,27/03/2020 18h25,1,o governar equador anunciar preparar cova cole...
3,Marfrig intensifica venda de carne do Brasil a...,https://economia.uol.com.br/noticias/reuters/2...,27/04/2020 14h53,1,o marfrig global foods retomar vender carnar b...
6,Coronavírus: sindicatos dos inquilinos convoca...,https://noticias.uol.com.br/internacional/ulti...,30/03/2020 10h38,1,greve pagamento aluguel organizar sindicato in...
9,\n\nVacina contra Covid-19 faz imunidade do co...,https://www.boatos.org/saude/vacina-contra-cov...,27/04/2021,0,“ reação vacinar entender facilidade pessoa in...
10,"Rio Grande do Norte recebe mais 81,7 mil doses...",https://www.gov.br/saude/pt-br/assuntos/notici...,22/03/2021 08h03,1,o ministério saudar coordenar sábado 20/03 e n...
...,...,...,...,...,...
11886,Coronavírus: Veja os seis requisitos da OMS pa...,https://noticias.uol.com.br/colunas/jamil-chad...,13/04/2020 16h06Atualizada em 13/04/2020 17h49,1,o debater intenso governo sair quarentenar e r...
11888,Real Madrid: Zidane testa positivo para a covi...,https://www.uol.com.br/esporte/futebol/ultimas...,22/01/2021 08h58,1,o real madrid informar manhã hoje o técnico zi...
11890,Mortes por covid-19 na Europa entre maiores de...,https://noticias.uol.com.br/saude/ultimas-noti...,15/04/2021 13h23,1,o proporção morte covid-19 europa idoso ano ca...
11891,\n\nVacinas estão sendo aplicadas com seringas...,https://www.boatos.org/saude/vacinas-aplicadas...,02/01/2021,0,atenção brasil simulação agulhar líquido conse...


In [6]:
# Combinar os DataFrames
df_final = pd.concat([df_true, df_false, df_fakerecogna], ignore_index=True)

display(df_final)

,title,link,date,corpo_texto,classe
0,Coronavirus,https://g1.globo.com/saude/coronavirus/,Data não disponível,g1 Coronavirus: Tudo sobre o COVID-19 Coronaví...,1
1,Novas doses da vacina contra mpox chegam este ...,https://g1.globo.com/saude/noticia/2024/11/12/...,Data não disponível,Novas doses da vacina contra mpox chegam este ...,1
2,VÍDEO: Agnaldo Rayol cantou 'Ave Maria' e 'Pai...,https://g1.globo.com/sp/sao-paulo/noticia/2024...,Data não disponível,VÍDEO: Agnaldo Rayol cantou 'Ave Maria' e 'Pai...,1
3,"Após afetar Tóquio, atletas testam positivo pa...",https://g1.globo.com/mundo/olimpiadas/paris-20...,Data não disponível,"Após afetar Tóquio, atletas testam positivo pa...",1
4,O que muda na campanha de vacinação contra cov...,https://g1.globo.com/saude/noticia/2024/06/01/...,Data não disponível,O que muda na campanha de vacinação contra cov...,1
...,...,...,...,...,...
3325,Coronavírus: Veja os seis requisitos da OMS pa...,https://noticias.uol.com.br/colunas/jamil-chad...,13/04/2020 16h06Atualizada em 13/04/2020 17h49,o debater intenso governo sair quarentenar e r...,1
3326,Real Madrid: Zidane testa positivo para a covi...,https://www.uol.com.br/esporte/futebol/ultimas...,22/01/2021 08h58,o real madrid informar manhã hoje o técnico zi...,1
3327,Mortes por covid-19 na Europa entre maiores de...,https://noticias.uol.com.br/saude/ultimas-noti...,15/04/2021 13h23,o proporção morte covid-19 europa idoso ano ca...,1
3328,\n\nVacinas estão sendo aplicadas com seringas...,https://www.boatos.org/saude/vacinas-aplicadas...,02/01/2021,atenção brasil simulação agulhar líquido conse...,0


In [7]:
# ----------------------------- PRÉ PROCESSAMENTO ---------------------------- #
# Garante que há conteúdo na coluna de título, corpo do texto e classe
df_final = df_final.dropna(subset=['title', 'corpo_texto', 'classe'])

# Garante que não há notícias repetidas
df_final = df_final.drop_duplicates()
df_final = df_final.reset_index(drop=True)

# Remove maiúsculas
df_final['corpo_texto'] = df_final['corpo_texto'].str.lower()
df_final['title'] = df_final['title'].str.lower()

# Tokeniza
df_final['corpo_texto'] = df_final['corpo_texto'].apply(lambda x: nltk.word_tokenize(x, language='portuguese'))
df_final['title'] = df_final['title'].apply(lambda x: nltk.word_tokenize(x, language='portuguese'))

# Remove stop_words e pontuação
stop_words = set(stopwords.words('portuguese'))

def clean_text(tokens):
    return [re.sub(r'[^\w\s]', '', word) for word in tokens if word not in stop_words and re.sub(r'[^\w\s]', '', word)]

df_final['corpo_texto'] = df_final['corpo_texto'].apply(clean_text)
df_final['title'] = df_final['title'].apply(clean_text)

# Embaralhar e resetar o índice
df_final = shuffle(df_final).reset_index(drop=True)

display(df_final)


,title,link,date,corpo_texto,classe
0,"[cai, número, casos, graves, coronavírus, frança]",https://noticias.uol.com.br/ultimas-noticias/e...,23/06/2020 03h33,"[parir, jun, efe, frança, anunciar, n, segunda...",1
1,"[aviões, pandemia, pátio, aeroporto, vira, gra...",https://economia.uol.com.br/todos-a-bordo/2020...,08/05/2020 04h00,"[cancelamento, voo, causar, pandemia, covid19,...",1
2,"[ladrão, roubou, 98, doses, coronavac, sp, pla...",https://noticias.uol.com.br/saude/ultimas-noti...,25/03/2021 13h11,"[homem, roubar, dose, vacinar, coronavac, covi...",1
3,"[sinovac, pagou, propina, doria, trazer, vacin...",https://www.boatos.org/saude/sinovac-pagou-pro...,09/12/2020,"[ih, enzuh, trazer, novidade, mídia, hehe, dit...",0
4,"[ofíciosacesse, ofícios, eventos, adversos, pó...",https://www.gov.br/saude/pt-br/vacinacao/esavi...,Data não disponível,"[ofícios, ministério, saúde, ir, conteúdo, 1, ...",1
...,...,...,...,...,...
3320,"[rio, janeiro, recebeu, imunizantes, contra, c...",https://www.gov.br/saude/pt-br/assuntos/notici...,Data não disponível,"[rio, janeiro, recebeu, imunizantes, contra, c...",1
3321,"[estratégia, vacinação, contra, covid19]",https://www.gov.br/saude/pt-br/vacinacao/publi...,Data não disponível,"[estratégia, vacinação, contra, covid19, minis...",1
3322,"[indonésia, inicia, vacinação, contra, covid19...",https://extra.globo.com/noticias/coronavirus/i...,Publicado em 13/01/21 11:12,"[indonésio, campanha, vacinação, covid19, n, q...",1
3323,"[fake, foto, mostre, corpos, idosos, covid19, ...",https://g1.globo.com/fato-ou-fake/coronavirus/...,25/04/2020,"[circular, rede, social, foto, homem, corpo, j...",0


In [8]:
# Salvar o df final
csv_file_path = "dados/df_final.csv"

compression_options = dict(method='zip', archive_name=csv_file_path)
df_final.to_csv(f'dados/df_final.zip', index=False, encoding='utf-8', quoting=csv.QUOTE_ALL, escapechar='\\', compression=compression_options)